Laboratorio 3 Noemí Pino (Proceso ETL)

FASE 1: EXTRACCIÓN

Fuente: Dataset público de Kaggle: Dirty Cafe Sales Dataset
Archivo: dirty_cafe_sales.csv
Registros: 10.000
Columnas: 8

Justificación de la fuente

Elegí este dataset porque:

    1. Tiene varios registros y lo más parecido a un registro real  

    2. Es un dataset público y sintético, diseñado específicamente para practicar limpieza y transformación (ETL).

    3. Contiene valores faltantes, inválidos e inconsistentes, lo que lo hace ideal para demostrar manejo de datos sucios, imputaciones, estandarización y enriquecimiento.S

    4. Su estructura (ventas de cafetería) es fácilmente entendible y aplicable a entornos reales de negocio.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("dirty_cafe_sales.csv") #leer el archivo CSV

#Revisar estructura general
print("Filas y columnas:", df.shape)        # tamaño del dataset
print("\nTipos de datos por columna:\n", df.dtypes)

#Ver las primeras filas
print("\nPrimeras filas del dataset:")
print(df.head(5))

#Ver cantidad de valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

#Comprobar si existen valores atípicos comunes ("ERROR", "UNKNOWN")
for col in df.columns:
    print(f"\nValores más comunes en {col}:")
    print(df[col].value_counts(dropna=False).head(5))


Filas y columnas: (10000, 8)

Tipos de datos por columna:
 Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

Primeras filas del dataset:
  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  

Valores nulos 

FASE 2: TRANSFORMACIÓN

Objetivo: Limpiar y transformar los datos del dataset para:

1. Eliminar o reemplazar valores inválidos (ERROR, UNKNOWN, vacíos).

2. Normalizar tipos de datos (numéricos, fechas).

3. Corregir totales inconsistentes (Total Spent).

4. Crear nuevas columnas (enriquecimiento).

5. Dejar los datos listos para cargarse a MongoDB.

In [3]:

#PASO 1: REEMPLAZO DE VALORES INVÁLIDOS

#Reemplazamos 'ERROR', 'UNKNOWN' y espacios vacíos por NaN
df = df.replace(["ERROR", "UNKNOWN", " "], pd.NA)

#Verificamos el cambio
print(df.isna().sum())


Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64


In [4]:
#PASO 2: CONVERSIÓN DE TIPOS

#Convertimos columnas numéricas a tipo float
num_cols = ["Quantity", "Price Per Unit", "Total Spent"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

#Revisamos los nuevos tipos
print(df[num_cols].dtypes)


Quantity          float64
Price Per Unit    float64
Total Spent       float64
dtype: object


In [5]:
#PASO 3: CORRECCIÓN DE TOTAL SPENT

#Si hay valores válidos en Quantity y Price, recalculamos el total
df["Calculated Total"] = df["Quantity"] * df["Price Per Unit"]

#Rellenamos Total Spent cuando está vacío
df["Total Spent"] = df["Total Spent"].fillna(df["Calculated Total"])

#Verificamos cuántos valores se corrigieron
print("Totales corregidos:", df["Total Spent"].isna().sum())


Totales corregidos: 40


In [6]:
# PASO 4: LIMPIEZA Y DERIVACIÓN DE FECHAS

#Convertir la columna a datetime (los inválidos se vuelven NaT)
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

#Crear columnas derivadas para análisis
df["Transaction Month"] = df["Transaction Date"].dt.to_period("M").astype(str)
df["Day of Week"] = df["Transaction Date"].dt.day_name()

#Mostrar ejemplo
print(df[["Transaction Date", "Transaction Month", "Day of Week"]].head())


  Transaction Date Transaction Month Day of Week
0       2023-09-08           2023-09      Friday
1       2023-05-16           2023-05     Tuesday
2       2023-07-19           2023-07   Wednesday
3       2023-04-27           2023-04    Thursday
4       2023-06-11           2023-06      Sunday


In [7]:
#PASO 5: ELIMINAR DUPLICADOS

#Eliminamos duplicados basados en Transaction ID
df = df.drop_duplicates(subset="Transaction ID")

# Confirmamos cantidad de filas después
print("Filas después de eliminar duplicados:", len(df))


Filas después de eliminar duplicados: 10000


In [8]:
#PASO 6: ENRIQUECIMIENTO DE DATOS

#Mapa de precios conocidos
price_map = {
    "Coffee": 2.0, "Tea": 1.5, "Sandwich": 4.0, "Salad": 5.0,
    "Cake": 3.0, "Cookie": 1.0, "Smoothie": 4.0, "Juice": 3.0
}

#Asignar precio esperado
df["Expected Price"] = df["Item"].map(price_map)

#Calcular desviación de precio
df["Price Deviation"] = df["Price Per Unit"] - df["Expected Price"]

#Crear categoría de producto (bebida / comida)
df["Category"] = df["Item"].apply(lambda x: "Beverage" if pd.notna(x) and x in ["Coffee", "Tea", "Smoothie", "Juice"] else "Food")

#Mostrar ejemplo
print(df[["Item", "Expected Price", "Price Per Unit", "Price Deviation", "Category"]].head(10))


       Item  Expected Price  Price Per Unit  Price Deviation  Category
0    Coffee             2.0             2.0              0.0  Beverage
1      Cake             3.0             3.0              0.0      Food
2    Cookie             1.0             1.0              0.0      Food
3     Salad             5.0             5.0              0.0      Food
4    Coffee             2.0             2.0              0.0  Beverage
5  Smoothie             4.0             4.0              0.0  Beverage
6      <NA>             NaN             3.0              NaN      Food
7  Sandwich             4.0             4.0              0.0      Food
8       NaN             NaN             3.0              NaN      Food
9  Sandwich             4.0             4.0              0.0      Food


In [9]:
#PASO 7: EXPORTAR DATASET LIMPIO

#Eliminamos columna auxiliar
df = df.drop(columns=["Calculated Total"])

#Guardar dataset limpio
df.to_csv("clean_cafe_sales.csv", index=False)

print("Archivo limpio exportado correctamente como 'clean_cafe_sales.csv'")


Archivo limpio exportado correctamente como 'clean_cafe_sales.csv'


FASE 3: CARGA A MONGODB ATLAS

Cargar los datos transformados desde el dataFrame a MongoDB Atlas, en una colección llamada cafe_sales_clean, dentro de una base de datos llamada etl_cafe.

In [ ]:

from pymongo import MongoClient

#Conexión cluster Atlas
uri = "mongodb+srv:CENSURADO=ClusterDataSciencieLAB"

#Crear cliente y base de datos
client = MongoClient(uri)
db = client["etl_cafe"]
collection = db["cafe_sales_clean"]

#Cargar el dataset limpio
clean_df = pd.read_csv("clean_cafe_sales.csv")

#Convertir dataFrame a lista de diccionarios
data_dict = clean_df.to_dict("records")

#Insertar datos en MongoDB
if data_dict:
    result = collection.insert_many(data_dict)
    print(f"{len(result.inserted_ids)} documentos insertados correctamente")
else:
    print("No hay datos para insertar")

#Verificar la inserción
print(f"Total documentos en colección: {collection.count_documents({})}")

# Mostrar un documento de ejemplo
print("Documento de ejemplo:")
print(collection.find_one())


10000 documentos insertados correctamente
Total documentos en colección: 10000
\n Documento de ejemplo:
{'_id': ObjectId('68f403225068f2321f61d093'), 'Transaction ID': 'TXN_1961373', 'Item': 'Coffee', 'Quantity': 2.0, 'Price Per Unit': 2.0, 'Total Spent': 4.0, 'Payment Method': 'Credit Card', 'Location': 'Takeaway', 'Transaction Date': '2023-09-08', 'Transaction Month': '2023-09', 'Day of Week': 'Friday', 'Expected Price': 2.0, 'Price Deviation': 0.0, 'Category': 'Beverage'}
